# [1045. Customers Who Bought All Products](https://leetcode.com/problems/customers-who-bought-all-products/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Customer

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| customer_id | int     |
| product_key | int     |
+-------------+---------+</pre>
This table may contain duplicates rows.
customer_id is not NULL.
product_key is a foreign key (reference column) to Product table.


Table: Product

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_key | int     |
+-------------+---------+</pre>
product_key is the primary key (column with unique values) for this table.


Write a solution to report the customer ids from the Customer table that bought all the products in the Product table.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Customer table:
<pre>+-------------+-------------+
| customer_id | product_key |
+-------------+-------------+
| 1           | 5           |
| 2           | 6           |
| 3           | 5           |
| 3           | 6           |
| 1           | 6           |
+-------------+-------------+</pre>
Product table:
<pre>+-------------+
| product_key |
+-------------+
| 5           |
| 6           |
+-------------+</pre>
Output:
<pre>+-------------+
| customer_id |
+-------------+
| 1           |
| 3           |
+-------------+</pre>
Explanation:
The customers who bought all the products (5 and 6) are customers with IDs 1 and 3.

In [2]:
# pandas schema

import pandas as pd

data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype(
    {'customer_id': 'Int64', 'product_key': 'Int64'})
data = [[5], [6]]
products = pd.DataFrame(data, columns=['product_key']).astype({'product_key': 'Int64'})

#converting to spark dataframe
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

customer_df = spark.createDataFrame(customer)
product_df = spark.createDataFrame(products)

In [3]:
customer

,customer_id,product_key
0,1,5
1,2,6
2,3,5
3,3,6
4,1,6


In [4]:
products

,product_key
0,5
1,6


In [5]:
# solving in spark dataframe
k=product_df.count()
customer_df\
    .groupBy('customer_id').agg(count_distinct('product_key').alias('pk'))\
    .where(col('pk')==k).show()

+-----------+---+
|customer_id| pk|
+-----------+---+
|          1|  2|
|          3|  2|
+-----------+---+



In [6]:
# solving in spark SQL

customer_df.createOrReplaceTempView('customer')
product_df.createOrReplaceTempView('product')

spark.sql('''

select customer_id from customer
group by customer_id
having count(distinct product_key)=(select count(product_key) from product)

''').show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+



In [7]:
spark.stop()